## _*FermionicOperator and qubit mapping*_

When we compute a FermionicOperator in Qiskit Chemistry it needs to be converted to a qubit operator to run on the simulator or real device. The FermionicOperator is built from electronn integrals where electrons behave anti-symmetrically under swap. qubits however do not exhibit this behavior and hence a mapping is needed to ensure that this is accounted for.

Here we have the jordan wigner mapping, the bravyi-kitaev mapping and a parity.

This notebook has been written to use the PYQUANTE chemistry driver. See the PYQUANTE chemistry driver readme if you need to install the external PyQuante2 library that this driver requires.

In [1]:
import numpy as np

from qiskit import BasicAer

from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.circuit.library import TwoLocal

from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PyQuanteDriver, UnitsType, BasisType

aqua_globals.random_seed = 50

In [2]:
# using driver to get fermionic Hamiltonian
# PyQuante example
driver = PyQuanteDriver(atoms='H .0 .0 .0; H .0 .0 0.735', units=UnitsType.ANGSTROM,
                        charge=0, multiplicity=1, basis=BasisType.BSTO3G)
molecule = driver.run()
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals

In [3]:
# convert from fermionic hamiltonian to qubit hamiltonian
ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_pa = ferOp.mapping(map_type='PARITY', threshold=0.00000001)
qubitOp_bk = ferOp.mapping(map_type='BRAVYI_KITAEV', threshold=0.00000001)

In [4]:
# print out qubit hamiltonian in Pauli terms and exact solution
qubit_ops = [(qubitOp_jw, 'jordan wigner'),
            (qubitOp_pa, 'parity'),
            (qubitOp_bk, 'bravyi-kitaev')]

for qubit_op, name in qubit_ops:
    qubit_op.chop(10**-10)

    print("\n --- {} ---".format(name))
    print(qubit_op)

    # Using exact eigensolver to get the smallest eigenvalue
    exact_eigensolver = NumPyMinimumEigensolver(qubit_op)
    ret = exact_eigensolver.run()
    print('The exact ground state energy using {} mapping is: {}'.format(name, ret.eigenvalue.real))    


 --- jordan wigner ---
Representation: paulis, qubits: 4, size: 15
The exact ground state energy using jordan wigner mapping is: -1.857275076637873

 --- parity ---
Representation: paulis, qubits: 4, size: 15
The exact ground state energy using parity mapping is: -1.857275076637872

 --- bravyi-kitaev ---
Representation: paulis, qubits: 4, size: 15
The exact ground state energy using bravyi-kitaev mapping is: -1.8572750766378743


Now we run on quantum backend, in this case a simulator

In [5]:
for qubit_op, name in qubit_ops:
    # setup VQE 
    # setup optimizer, use L_BFGS_B optimizer for example
    lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

    # setup variational form generator (generate trial circuits for VQE)
    var_form = TwoLocal(qubit_op.num_qubits, 'ry', 'cz', reps=5, entanglement='full')

    # setup VQE with operator, variational form, and optimizer
    vqe_algorithm = VQE(qubit_op, var_form, lbfgs)

    backend = BasicAer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend)

    results = vqe_algorithm.run(quantum_instance)

    print("Ground state energy using {}: {}".format(name, results.eigenvalue.real))

Ground state energy using jordan wigner: -1.85727476869041
Ground state energy using parity: -1.857274320675
Ground state energy using bravyi-kitaev: -1.85726132279563
